In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter CRUD Python module file name and class name
from aacConnect import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password for accessing database
username = "aacuser2"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(
    username, 
    password, 
    host='nv-desktop-services.apporto.com', 
    port=30702,
    database_name='AAC',
    collection_name='animals'
)

# get initial data set
initial_data = db.read({})
# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(initial_data)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# set up app format
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # add logo with link to snhu website
    html.A( [html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()), 
        height=150, width=150)], href='https://www.snhu.edu'),
    
    # header and unique identifier
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Danielle Eeg 2-24-2024'))),
    html.Hr(),
    html.Div(
        
        # Add radio buttons for selecting a filter type
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline'}  # Display each radio button on a separate line
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        
        data=df.to_dict('records'),
        # Allow cells to be editable
        editable=True,
        
        # set sortint and filtering
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        filter_action="native",
        
        # page set-up:  10 items/page, starting on page 0
        page_size=5,
        page_current=0,
        
        # Allow single selection of columns and rows
        row_selectable="single",
        column_selectable='single',
        
        # Allow end-user to delete rows
        row_deletable=True,
        
        # at time of page load, select the first row
        selected_rows=[0]

                        ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so the pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter Options
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])  

# Method for updating data table when filter applied (radio buttons)              
def update_datatable(filter_type):  
    
    # filter for reset selection
    if filter_type == "reset":
        df = pd.DataFrame.from_records(db.read({}))
        
    # filter for water rescue selection
    elif filter_type == "water":
        df = pd.DataFrame.from_records(db.read(
            { '$and': [ 
                {'animal_type':'Dog'},
                { 'breed': 
                       { '$in': [
                           "Labrador Retriever Mix", 
                           "Chesapeake Bay Retriever", 
                           "Newfoundland"
                       ] } }, 
                { 'sex_upon_outcome': 'Intact Female' },
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$lte':156}}, 
                    {'age_upon_outcome_in_weeks':{'$gte':26}}
                ]}
            ]}
        ))
    
    # filter for mountain and wilderness rescue
    elif filter_type == "mountain":
        df = pd.DataFrame.from_records(db.read(
            { '$and': [ 
                {'animal_type':'Dog'},
                { 'breed': 
                       { '$in': [
                           "German Shepard", 
                           "Alaskan Malamute", 
                           "Old English Sheepdog",
                           "Siberian Husky",
                           "Rottweiler"
                       ] } }, 
                { 'sex_upon_outcome': 'Intact Male' },
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$lte':156}}, 
                    {'age_upon_outcome_in_weeks':{'$gte':26}}
                ]}
            ]}
        ))

    # filter for disaster selection    
    elif filter_type == "disaster":
        df = pd.DataFrame.from_records(db.read(
            { '$and': [
                {'animal_type':'Dog'},
                { 'breed': { 
                    '$in': [
                           "German Shepard", 
                           "Doberman Pinscher", 
                           "Golden Retriever",
                           "Bloodhound",
                           "Rottweiler"
                       ] } }, 
                { 'sex_upon_outcome': 'Intact Male' },
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$lte':300}}, 
                    {'age_upon_outcome_in_weeks':{'$gte':20}}
                ]}
            ] 
            }
        ))
        
    # Update columns ased on filter selection
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)
   

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"), 
    [Input('datatable-id', "derived_virtual_data")]
)

# Update Pie Chart
def update_graphs(viewData):
    
    # verify data exists
    if viewData is None:
        return
    
    # set chart data
    df_chart = pd.DataFrame.from_dict(viewData)
    
    # return Pie Chart
    return [
        dcc.Graph(            
            figure = px.pie(df_chart, 
                            names=df_chart.breed, 
                            values=df_chart.age_upon_outcome_in_weeks, 
                            title='Preferred Animals'
                           )
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

# function for updating style based on selection
def update_styles(selected_columns):
    
    # check for no selected columns
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    
    # Check for existance of data
    if viewData is None:
        return
    elif index is None:
        return
    
    # get data and use as dataframe
    dff = pd.DataFrame.from_dict(viewData)
    
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or index[0]>=len(dff):
        row = 0
    else: 
        row = index[0]
        
    # Use reference location Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:30090/
